# All Data

In [2]:
f1 = load_metric("f1")
recall = load_metric("recall")
precision =  load_metric("precision")
def preprocess_function(examples, tok):
    return tok(examples["text"], truncation=True, max_length=512)


def compute_metrics(p):    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    metric = f1.compute(predictions=predictions, references=labels, average="macro")
    metric.update(recall.compute(predictions=predictions, references=labels, average="macro"))
    metric.update(precision.compute(predictions=predictions, references=labels, average="macro"))
    return metric
data = getData(sub_task = f"A", return_type = "dataset", pre_proccessed = True)
tokenizer = AutoTokenizer.from_pretrained("/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_data = data.map(preprocess_function,fn_kwargs = {'tok':tokenizer}, batched=True)

100%|██████████| 2/2 [00:00<00:00, 559.05it/s]


In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 44324
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1270
    })
})

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2", num_labels=2)
training_args = TrainingArguments(
    output_dir="/scratch/mt/ashapiro/Hate_Speech/All_data_run/",
    learning_rate=2e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=100,
    weight_decay=0.001,
    load_best_model_at_end=True,
    group_by_length = True, 
    seed = 20,
    report_to = 'none'
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

Some weights of the model checkpoint at /scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequence

In [5]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ashapiro/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 44324
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 277100
  0%|          | 501/277100 [00:34<5:49:27, 13.19it/s]

{'loss': 0.4942, 'learning_rate': 1.9963911945146156e-05, 'epoch': 0.18}


  0%|          | 1001/277100 [01:09<5:21:08, 14.33it/s]

{'loss': 0.4453, 'learning_rate': 1.9927823890292315e-05, 'epoch': 0.36}


  1%|          | 1501/277100 [01:42<5:35:32, 13.69it/s]

...

 11%|█         | 30481/277100 [34:39<3:51:56, 17.72it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4

 97%|█████████▋| 307/318 [00:04<00:00, 76.15it/s]
                                                        
100%|██████████| 318/318 [00:04<00:00, 76.49it/s]
                                                 Saving model checkpoint to /scratch/mt/ashapiro/Hate_Speech/All_data_run/checkpoint-30481
Configuration saved in /scratch/mt/ashapiro/Hate_Speech/All_data_run/checkpoint-30481/config.json


{'eval_loss': 1.3483388423919678, 'eval_f1': 0.7861308871851042, 'eval_recall': 0.800525346991974, 'eval_precision': 0.7781972990707617, 'eval_runtime': 4.3151, 'eval_samples_per_second': 294.316, 'eval_steps_per_second': 73.695, 'epoch': 11.0}


Model weights saved in /scratch/mt/ashapiro/Hate_Speech/All_data_run/checkpoint-30481/pytorch_model.bin
tokenizer config file saved in /scratch/mt/ashapiro/Hate_Speech/All_data_run/checkpoint-30481/tokenizer_config.json
Special tokens file saved in /scratch/mt/ashapiro/Hate_Speech/All_data_run/checkpoint-30481/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /scratch/mt/ashapiro/Hate_Speech/All_data_run/checkpoint-2771 (score: 0.8346165439057007).
 11%|█         | 30481/277100 [34:47<4:41:32, 14.60it/s]

{'train_runtime': 2087.8921, 'train_samples_per_second': 2122.907, 'train_steps_per_second': 132.718, 'train_loss': 0.13503938274884772, 'epoch': 11.0}


TrainOutput(global_step=30481, training_loss=0.13503938274884772, metrics={'train_runtime': 2087.8921, 'train_samples_per_second': 2122.907, 'train_steps_per_second': 132.718, 'train_loss': 0.13503938274884772, 'epoch': 11.0})